<h1 style="text-align: center;"> Preprocesamiento </h1>
<h1 style="text-align: right; font-size: 24px; margin-right: 10px;">Guillermo Díaz Aguado</h1>

## Introducción
Uno de los factores más importantes a la hora de crear nuestro modelo de aprendizaje es que nuestros datos tengan información de calidad y una buena cantidad. Es por ello que vamos a indicar una serie de pasos que se deberían hacer *SIEMPRE* antes de empezar a crear nuestro modelo. 

## Gestión de valores perdidos
A la hora de recopilar datos pueden ocurrir muchos fallos (o no tienen por qué ser fallos) que hagan que nuestro dataset tengan valores perdidos, nulos. Estos datos se pueden deber a decenas de factores como: 
| Tipo de dato faltante | Descripción |
|------------------------|-------------|
| Simplemente se han perdido aleatoriamente al llegar a nuestra mano | Generalmente debido a la aleatoriedad, con lo que no afectan en gran medida |
| Se han dejado sin contestar | En casos como estos nos pueden dar información extra, ya que es posible que se hayan dejado sin contestar debido a factores que influyen sobre la variable dependiente. Una buena solución a estos elementos es crear una nueva variable dicotómica que nos indique si han dejado sin contestar o no dicho espacio |

Desafortunadamente, estos valores son casi imposibles de manejar para la mayoria de las herramientas computacionales, es por ello que debemos tratarlas antes. Los valores perdidos se pueden mostrar de infinitas formas, lo optimo sería que nos diesen alguna pista al recibir el datset, pero sabemos que eso muchas veces es imposible, por lo que muestro algunos ejemplos:
* Valor NaN
* Valor null
* Celda vacía
* Celda con caracteres en una columna con valores numericos
* Caracteres como "?"
* Valores excesivamente altos como 9999 o valores extraños como 0 o -1


## Escalado de las variables independientes.
El escalado de los datos es un punto tan importante, que para algunos modelos estadisticos es NECESARIO aplicar una transformación de escalado para que este algoritmo funcione.

Prácticamente siempre habrá que usar un escalado, que de manera general, no empeora el modelo. Pero, ¿Para que sirve?
    Pongamonos en el caso que queremos hacer un modelo con estos rangos de variables independientes: $X_1:[0-100]$ y $X_2:[1.000-1.000.000]$.
    Seguramente la variable $X_2$ tendrá un mayor peso en el modelo, debido únicamente a la escala de esta variable y no debido a la posible correlación de la variable predictora $X_2$ con la variable dependiente Y.




$\textcolor{blue}{Este texto es azul}$
<p style="color: red;">Este texto es rojo</p>
